In [6]:
from math import pi, sqrt, cosh, exp, floor
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad

In [16]:
HBARC = 197.3269718
NEUTRON_MASS = 939.5653 / HBARC
PROTON_MASS = 938.272 / HBARC
NUCLEON_MASS = 2 * NEUTRON_MASS * PROTON_MASS / (NEUTRON_MASS + PROTON_MASS)
MUON_MASS = 105.7 / HBARC
ELECTRON_MASS = 0.511 / HBARC
GA = 1.267
MEVFM3_TO_GCM3 = 1.78e12
FM4_TO_DYNECM2 = 3.16e35
EG_TO_MEV = 7.69**2 / 10**16

COS_TC = sqrt(0.95)
GF = (HBARC / 292800)**2
GN = -1.913 * 2
GP = 3.586 + 2

def gtild(eb, t):
    return GF**2 * COS_TC**2 * eb / (4 * pi * cosh(GN * eb / (NUCLEON_MASS * t)) * cosh(GP * eb / (4 * NUCLEON_MASS * t)))
def gtildpr(eb, t):
    return GF**2 * COS_TC**2 * eb / (4 * pi * cosh(GN * eb / (NUCLEON_MASS * t)) * cosh((GP - 2) * eb / (4 * NUCLEON_MASS * t)))
def nfd(e, mu, t):
    return 1 / (np.exp((e - mu) / t) + 1)

In [ ]:
def electron_energy(eb, n, kz):
    return np.sqrt(2 * n * eb + kz**2 + ELECTRON_MASS**2)

def electron_density(mu, eb, t):
    if mu < 0:
        max_ll = floor(10 * t**2 / (2 * eb))
    else:
        max_ll = floor(10 * (t**2 + mu) / (2 * eb))

    pos_ll = np.linspace(1, max_ll, max_ll)
    return quad(lambda kz: eb / (2 * pi**2) * (1 + 2 * np.sum(nfd(electron_energy(eb, pos_ll, kz), mu, t))), 0, max((10 * t, 10 * (t + mu))))[0]

def mue_solver(ne, eb, t):
    return fsolve(lambda mu: ne - electron_density(mu, eb, t), t)

In [ ]:
#if ne is 0, set ne0 to 1
def mat_elt(sp, sn, ct_nu, ne0 = 0):
    if sp == 1 and sn == 1:
        return 2 * (1 + GA)**2 * (1 + ct_nu) + 2 * (1 - GA)**2 * (1 - ct_nu) * (1 - ne0)
    elif sp == 1 and sn == -1:
        return 8 * GA**2 * (1 - ct_nu)
    elif sp == -1 and sn == 1:
        return 8 * GA**2 * (1 + ct_nu) * (1 - ne0)
    elif sp == -1 and sn == -1:
        return 2 * (1 - GA)**2 * (1 + ct_nu) + 2 * (1 + GA)**2 * (1 - ct_nu) * (1 - ne0)
    else:
        print('invalid spin')
        return 0

In [ ]:
def kappan_high(nb, yp, eb, t, knu, ct_nu):